# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt



def cubic_interpolation(image, scale_y, scale_x, A_invert):
    height, width = image.shape
    new_heigth = int((scale_y) * height)
    new_width = int((scale_x) * width)
    new_image = np.zeros((new_heigth, new_width), np.uint8)
    for j in range(new_heigth):
        for i in range(new_width):

            i_original = i / scale_x
            j_original = j / scale_y
            
            i1 = int(np.floor(i_original))
            i2 = min(i1 + 1, width - 1)
            j1 = int(np.floor(j_original))
            j2 = min(i2 + 1, height - 1)
            
            if i2 >= width - 1 or j2 >= height - 1:
                continue

            A = image[j1, i1]
            B = image[j2, i1]
            C = image[j1, i2]
            D = image[j2, i2]
                
            Ax = (image[j1, min(i1 + 1, width - 1)] - image[j1, max(i1 - 1, 0)]) / 2
            Bx = (image[j2, min(i1 + 1, width - 1)] - image[j2, max(i1 - 1, 0)]) / 2
            Cx = (image[j1, min(i2 + 1, width - 1)] - image[j1, max(i2 - 1, 0)]) / 2
            Dx = (image[j2, min(i2 + 1, width - 1)] - image[j2, max(i2 - 1, 0)]) / 2

            Ay = (image[min(j1 + 1, height - 1), i1] - image[max(j1 - 1, 0), i1]) / 2
            By = (image[min(j2 + 1, height - 1), i1] - image[max(j2 - 1, 0), i1]) / 2
            Cy = (image[min(j1 + 1, height - 1), i2] - image[max(j1 - 1, 0), i2]) / 2
            Dy = (image[min(j2 + 1, height - 1), i2] - image[max(j2 - 1, 0), i2]) / 2

            Axy = (image[min(j1 + 1, height - 1), min(i1 + 1, width - 1)] - image[max(j1 - 1, 0), min(i1 + 1, width - 1)] - image[min(j1 + 1, height - 1), max(i1 - 1, 0)] + image[max(j1 - 1, 0), max(i1 - 1, 0)]) / 4
            Bxy = (image[min(j2 + 1, height - 1), min(i1 + 1, width - 1)] - image[max(j2 - 1, 0), min(i1 + 1, width - 1)] - image[min(j2 + 1, height - 1), max(i1 - 1, 0)] + image[max(j2 - 1, 0), max(i1 - 1, 0)]) / 4
            Cxy = (image[min(j1 + 1, height - 1), min(i2 + 1, width - 1)] - image[max(j1 - 1, 0), min(i2 + 1, width - 1)] - image[min(j1 + 1, height - 1), max(i2 - 1, 0)] + image[max(j1 - 1, 0), max(i2 - 1, 0)]) / 4
            Dxy = (image[min(j2 + 1, height - 1), min(i2 + 1, width - 1)] - image[max(j2 - 1, 0), min(i2 + 1, width - 1)] - image[min(j2 + 1, height - 1), max(i2 - 1, 0)] + image[max(j2 - 1, 0), max(i2 - 1, 0)]) / 4

            x = [A, B, D, C, Ax, Bx, Dx, Cx, Ay, By, Dy, Cy, Axy, Bxy, Dxy, Cxy]
            
            a = np.dot(A_invert, x)
    
            new_image[j, i] = int(np.clip(a[0], 0, 255))

    return new_image

A_invert = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
                     [-3,  3,  0,  0, -2, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
                     [2, -2,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
                     [0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
                     [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
                     [0,  0,  0,  0,  0,  0,  0,  0, -3,  3,  0,  0, -2, -1,  0,  0],
                     [0,  0,  0,  0,  0,  0,  0,  0,  2, -2,  0,  0,  1,  1,  0,  0],
                     [-3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0,  0,  0,  0,  0],
                     [0,  0,  0,  0, -3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0],
                     [9, -9, -9,  9,  6,  3, -6, -3,  6, -6,  3, -3,  4,  2,  2,  1],
                     [-6,  6,  6, -6, -3, -3,  3,  3, -4,  4, -2,  2, -2, -2, -1, -1],
                     [2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0],
                     [0,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0],
                     [-6,  6,  6, -6, -4, -2,  4,  2, -3,  3, -3,  3, -2, -1, -2, -1],
                     [4, -4, -4,  4,  2,  2, -2, -2,  2, -2,  2, -2,  1,  1,  1,  1],])

parrot = cv2.imread('parrot.bmp')
parrot = cv2.cvtColor(parrot, cv2.COLOR_BGR2GRAY)
parrot = cubic_interpolation(parrot, 3, 3, A_invert)

plt.figure(figsize=(parrot.shape[0]/100, parrot.shape[1]/100), dpi=200)
plt.imshow(parrot, cmap ="gray")
plt.axis('off')
plt.show()

